<h4>RNNs,LSTMs, GRU, MultiLayer RNNs, RNNs + Transformer </h4>
<h4>Self attention and multihead attention</h4>

<h4>Fine Tuning LLMS</h4>
<h4>Reinforcement Learning with Human Feedback</h4>
<p>ChatGPT does not use the log loss functions we are coding up. They supplement it with RL and other proprietary methods </p>
<img src="rlhf.png"/ >

</h4>RLHF Adjustments for Tractability</h4>
<h4>RL Proximal Policy Optimization Original Paper w code</h4>
<a href="https://arxiv.org/pdf/1707.06347">paper reference</a>
<h4>Gradient Policy Methods</h4>
<h5>Gradient Estimator</h5>
$$\hat{g} = \hat{E_t}[\nabla_{\theta} logP_{\theta}(\alpha_t | s_t)\hat{A_t}] $$
<h5>Estimating policy gradient Loss Fn: </h5>
$$L^{PG}(\theta) = \hat{E_t}[logP_{\theta}(\alpha_t | s_{t})\hat{A_t}]$$
<h5>Intractable bc large policy updates, add modification via TRPO</h5>
$$max_{\theta} E_{\theta}[\frac{P_{\theta}(\alpha_t | s_t)}{P_{old}(\alpha_t | s_t)}\hat{A_t}]$$

<h5>LCLIP</h5>
$$L^{CLIP}(\theta) = \hat{E_t}[min(r_t(\theta)\hat{A_t},clip(r_t{\theta}, 1-\epsilon, 1+epsilon)\hat{A_t})]$$
<a href="https://github.com/openai/baselines/tree/master/baselines">RL PPO</a>



<h4>RLHF with DPO</h4>
<h5>DeepLearningAI short course on RLHF, no code</h5>
<a href="https://learn.deeplearning.ai/courses/reinforcement-learning-from-human-feedback/lesson/2/how-does-rlhf-work"></a>

<img src="rlhf_gc1.png" />
<h4>There are 2 models being trained, a reward model and RL policy. </h4>
<img src="rlhf_gc2.png" />
<img src="rlhf_gc3.png" />
<img src="rlhf_gc4.png" />
<img src="rlhf_gc5.png" />


<h4>Runs on GCP Vertex AI pipeline</h4>
<ol>
<li>GCP source code not easy to access. Is it published? </li>
<li>Dataset link: https://huggingface.co/datasets/webis/tldr-17</li>
</ol>
<h4>1 day to run, 24h * 4$/hr? maybe 100$ to run it; if it works</h4>
<h4>Additional Resources/h4>
<ol>
<li> RL TRPO/PPO lecture https://www.youtube.com/watch?v=H8rElrvs9Lo</li>
<li>HW link: https://www.youtube.com/watch?v=4RyX7L-MbsU</li>
<li>https://github.com/upb-lea/reinforcement_learning_course_materials </li> 
</ol>

<h4>Many Steps in RLHF, eg SFT</h4>
<img src="rlhf_block.png">

<h4>Hugging Face RLHF pipeline = SFT+Reward Modeling+PPO step </h4>
<h4>SFT</h4>
<code>
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

dataset = load_dataset("imdb", split="train")

sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=512,
    output_dir="/tmp",
)
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=sft_config,
)
trainer.train()
</code>
<h4>Reward Modeling</h4>
<code>
from peft import LoraConfig, TaskType
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from trl import RewardTrainer, RewardConfig

model = AutoModelForSequenceClassification.from_pretrained("gpt2")
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

...

trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=dataset,
    peft_config=peft_config,
)

trainer.train()

</code>
<h4>PPO Step</h4>
<code>
python -i examples/scripts/ppo/ppo.py \
    --learning_rate 3e-6 \
    --num_ppo_epochs 1 \
    --num_mini_batches 1 \
    --output_dir models/minimal/ppo \
    --per_device_train_batch_size 64 \
    --gradient_accumulation_steps 1 \
    --total_episodes 10000 \
    --model_name_or_path EleutherAI/pythia-1b-deduped \
    --non_eos_penalty \
</code>
<img src="ppo_metrics.png" />

<h4>RL with DPO</h4>
<p>Paper Link: https://arxiv.org/abs/2305.18290</p>
<h4>RLHF with DPO:</h4>
<p>Code from HF: https://github.com/huggingface/blog/blob/main/pref-tuning.md</p>
<ol>
<li>There are 2 loss functions, a reward loss fn and a policy reward fn. </li>
<li>Soruce: https://web.stanford.edu/class/cs234/slides/dpo_slides.pdf</li>
</ol>
<img src="rl_dpo1.png" />
<img src="rl_dpo2.png" />


<h4>HF DPO Trainer</h4>
<code>
import torch
from trl import DPOConfig, DPOTrainer
from unsloth import FastLanguageModel

max_seq_length = 2048 # Supports automatic RoPE Scaling, so choose any number.

# Load model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/zephyr-sft",
    max_seq_length = max_seq_length,
    dtype = None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True, # Use 4bit quantization to reduce memory usage. Can be False.
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

# Do model patching and add fast LoRA weights
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Dropout = 0 is currently optimized
    bias = "none",    # Bias = "none" is currently optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
)

training_args = DPOConfig(
    output_dir="./output",
    beta=0.1,
)

dpo_trainer = DPOTrainer(
    model,
    ref_model=None,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)
dpo_trainer.train()
</code>

In [ ]:
<h4>Pytorch Kernels</h4>
<ol>
<li>CPU or GPU kernels different than using nn.Module. Parallelism not specified in nn.Module</li>
<li>A kernel manages the parallelism in a code block. CPU cores vs. thousands of GPU threads</li>
<li>Optimize for training means GPU Kernels</li>
<li>Strategy 1: NVIDIA cuBLAS using hand optimized CUDA code</li>
<li>Strategy 2: NVIDIA Cutlass: https://github.com/NVIDIA/cutlass</li>
<li>Strategy 3: OpenAI/Pytorch Triton</li>
</ol>

<h4>Simple approach to optimization</h4>
<p> Add torch compile<p>
<code>
    def train(mod, data):
        opt.zero_grad(True)
        pred = mod(data[0])
        loss = torch.nn.CrossEntropyLoss()(pred, data[1])
        loss.backward()
        opt.step()

    eager_times = []
    for i in range(N_ITERS):
        inp = generate_data(16)
        _, eager_time = timed(lambda: train(model, inp))
        eager_times.append(eager_time)
        print(f"eager train time {i}: {eager_time}")
    print("~" * 10)

    model = init_model()
    opt = torch.optim.Adam(model.parameters())
    train_opt = torch.compile(train, mode="reduce-overhead")
</code>


<h4>RLHF accuracy of 67 </h4>
https://huggingface.co/blog/stackllama


<h4>Raschka LLM </h4>
<p>
Links: https://sebastianraschka.com/blog/2024/research-papers-in-march-2024.html
</p>
<p>
Links: https://magazine.sebastianraschka.com/p/llm-training-rlhf-and-its-alternatives
</p>

<h4>Triton Compiler and Language</h4>
<ol>
<li>Triton language looks like assembly (load/store)</li>
<li>OpenAI/FB/Google have similar tools to recode from python to device native formats for efficiency. Est 2-4x speedup. Prevent data starvation for multiple jobs</li>
<li>Open sourced to try to get more contributions</li>   
<li>Inference requires triton</li>
<li>https://www.youtube.com/watch?v=Nh5QIkGuExQ</li>
</ol>

In [ ]:
<h4>Pytorch Cutlass</h4>
<p>https://www.youtube.com/watch?v=yCyZEJrlrfY</p>
Python

In [ ]:
<h4>FB tools</h4>
<h4>FB AI Template for inference serving.</h4> 
https://github.com/facebookincubator/AITemplate
<h4>Transformer templates from FB</h4>
https://github.com/facebookresearch/xformers
